In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta

In [57]:
#load sp500 data

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

end_date = '2023-09-27'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

sp500 = sp500[pd.to_datetime(sp500["Date added"])<start_date]

symbols_list = sp500['Symbol'].unique().tolist()

df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack(future_stack=True)

df.columns = df.columns.str.lower()

[*********************100%%**********************]  345 of 345 completed


In [58]:
#calculate features

df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2)/2 - (2*np.log(2)-1)*(np.log(df['adj close']) - np.log(df['open']))**2
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'], low=stock_data['low'], close=stock_data['close'], length=14)
    return atr.sub(atr.mean()).div(atr.std())
df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6


In [71]:
#aggregate to monthly level

last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]

data = pd.concat([df.unstack('Ticker')['dollar_volume'].resample('ME').mean().stack('Ticker', future_stack=True).to_frame('dollar_volume'),
           df.unstack()[last_cols].resample('ME').last().stack('Ticker', future_stack=True)], axis=1).dropna()

In [73]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('Ticker').rolling(5*12, min_periods=12).mean().stack())
data['dollar_vol_rank'] = (data.groupby('Date')['dollar_volume'].rank(ascending=False))
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

adj close  garman_klass_vol        rsi    bb_low  \
Date       Ticker                                                      
2016-10-31 AAL      39.134335         -0.000176  62.203563  3.604673   
           AAPL     26.182167         -0.002541  49.891057  3.293123   
           ABBV     39.878784         -0.049190  27.477728  3.744517   
           ABT      34.112480         -0.008074  38.008843  3.549492   
           ACN     103.117409         -0.005023  53.823715  4.633009   
...                       ...               ...        ...       ...   
2023-09-30 WFC      39.479488         -0.000317  40.920307  3.689633   
           WMT      53.445198         -0.000074  54.722543  3.982183   
           WYNN     89.675743          0.000157  36.293529  4.493434   
           XOM     112.466652         -0.000205  59.440202  4.679146   
           ZTS     175.131134          0.000049  42.623482  5.153746   

                     bb_mid   bb_high       atr      macd  
Date       Ticker                                          
2016-10-31 AAL     3.655494  3.706314  0.402199  1.131596  
           AAPL    3.322002  3.350880 -1.038688 -0.195978  
           ABBV    3.798670  3.852822 -0.893132 -0.760593  
           ABT     3.599959  3.650426 -1.035224 -0.650888  
           ACN     4.644646  4.656283 -0.996806 -0.135456  
...                     ...       ...       ...       ...  
2023-09-30 WFC     3.729990  3.770346 -0.558742 -0.282325  
           WMT     3.999651  4.017120 -0.196381  0.399459  
           WYNN    4.559234  4.625035 -1.348097 -0.693417  
           XOM     4.719239  4.759332  0.601335  1.400623  
           ZTS     5.212559  5.271371  0.651515 -1.188278  

[12516 rows x 8 columns]

In [74]:
#calculate mmonthly returns for different time horizons
def calculate_returns(df):
    outlier_cutoff = 0.005
    lags = [1,2,3,6,9,12]
    for lag in lags:
        df[f'return_{lag}m'] = (df['adj close']
                               .pct_change(lag)
                               .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                               .add(1)
                               .pow(1/lag)
                               .sub(1))
    return df
data = data.groupby('Ticker', group_keys=False).apply(calculate_returns).dropna()

In [75]:
#download fama franch factors and calculate rolling betas
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',start='2010')[0].drop('RF', axis =1 )
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample('M').last().div(100)
factor_data = factor_data.join(data['return_1m']).sort_index()

C:\Users\Thoma\AppData\Local\Temp\ipykernel_28040\668726976.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',start='2010')[0].drop('RF', axis =1 )
C:\Users\Thoma\AppData\Local\Temp\ipykernel_28040\668726976.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',start='2010')[0].drop('RF', axis =1 )
C:\Users\Thoma\AppData\Local\Temp\ipykernel_28040\668726976.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  factor_data = factor_data.resample('M').last().div(100)


In [79]:
observations = factor_data.groupby(level=1).size()
valid_stocks = observations[observations >= 10]
factor_data = factor_data[factor_data.index.get_level_values('Ticker').isin(valid_stocks.index)]

In [85]:
betas = (factor_data.groupby(level=1, group_keys=False)
.apply(lambda x: RollingOLS(endog=x['return_1m'],
                            exog=sm.add_constant(x.drop('return_1m', axis=1)),
                            window=min(24, x.shape[0]),
                            min_nobs=len(x.columns)+1)
 .fit()
 .params
 .drop('const', axis=1)))
       

In [92]:
factors = ['Mkt-RF', 'SMB',	'HML', 'RMW', 'CMA']
data = (data.join(betas.groupby('Ticker').shift()))



In [93]:
data.loc[:, factors] = data.groupby('Ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

In [96]:
data = data.dropna()
data = data.drop('adj close', axis=1)
data.info

<bound method DataFrame.info of                    garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
Date       Ticker                                                              
2017-10-31 AAL            -0.000363  41.051768  3.849110  3.921750  3.994389   
           AAPL           -0.001105  69.196669  3.593605  3.640476  3.687347   
           ABBV           -0.036142  55.247895  4.187696  4.234050  4.280405   
           ABT            -0.005677  53.844904  3.887384  3.910952  3.934519   
           ACN            -0.004274  69.365345  4.798335  4.838013  4.877691   
...                             ...        ...       ...       ...       ...   
2023-09-30 VZ             -0.001680  42.222476  3.451862  3.484054  3.516245   
           WFC            -0.000317  40.920307  3.689633  3.729990  3.770346   
           WMT            -0.000074  54.722543  3.982183  3.999651  4.017120   
           WYNN            0.000157  36.293529  4.493434  4.559234  4.625035   
        